In [37]:
import open3d as o3d
import numpy as np
import os

# Read the point cloud
###Changed Reading Route
pcd = o3d.io.read_point_cloud('C:/Users/jessi/OneDrive/Dokument/ProjectCourse/data/2024-09-02_130544_243301-11/raw/tray-b-4-c_L2.pcd')

#distances = pcd.compute_nearest_neighbor_distance()
#avg_dist = np.mean(distances)
#rint(f"Average distance between points: {avg_dist}")

#pcd = pcd.voxel_down_sample(voxel_size=2*avg_dist)
#len(pcd.points)

pcd = pcd.uniform_down_sample(10)
len(pcd.points)

# Convert to numpy arrays for easier manipulation
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)  # Get the colors of the point cloud

print('original chips color number is: ', colors.shape)
print('original chips points number is: ', points.shape)

min_bound = points.min(axis=0)
max_bound = points.max(axis=0)

ranges = max_bound - min_bound
longest_axis_index = np.argmax(ranges)  # Index of the longest axis

# Define midpoints for partitioning along the longest axis
midpoints = np.linspace(min_bound[longest_axis_index], max_bound[longest_axis_index], num=5)

print('midpoints', midpoints)

parts = [
    (points[(points[:, 1] < midpoints[1])], colors[(points[:, 1] < midpoints[1])]),  # Part 1
    (points[(points[:, 1] >= midpoints[1]) & (points[:, 1] < midpoints[2])], colors[(points[:, 1] >= midpoints[1]) & (points[:, 1] < midpoints[2])]),  # Part 2
    (points[(points[:, 1] >= midpoints[2]) & (points[:, 1] < midpoints[3])], colors[(points[:, 1] >= midpoints[2]) & (points[:, 1] < midpoints[3])]),  # Part 3
    (points[(points[:, 1] >= midpoints[3])], colors[(points[:, 1] >= midpoints[3])])]  # Part 4

# Visualize each part while preserving original colors
part_clouds = []

### Added in output folder as well as the folder for not qualified chips
output_dir = '/Users/jessi/OneDrive/Dokument/ProjectCourse/output'  # Replace this with your desired path
discard_dir = '/Users/jessi/OneDrive/Dokument/ProjectCourse/discard' # Replace this with your desired path

# Make sure the directory exists
#os.makedirs(output_dir, exist_ok=True)
#os.makedirs(discard_dir, exist_ok=True)

for i, (part_points, part_colors) in enumerate(parts):
    if part_points.size > 0:  # Only create a PointCloud if there are points in the part
        directory=[]
        
        min_bound = part_points.min(axis=0)
        max_bound = part_points.max(axis=0)
        ranges = max_bound - min_bound
        longest_axis_index = np.argmax(ranges)  # Index of the longest axis
        
        min_bound[longest_axis_index], max_bound[longest_axis_index]
        
        endpoints = np.linspace(min_bound[longest_axis_index], max_bound[longest_axis_index], num=2)
        
        uppernumberratio=0.15 #approximate ratio of how much room numbers take up
        lowernumberratio=0.07
        
        part_points2=part_points
        
        part_points = part_points[((endpoints[0]+(endpoints[1]-endpoints[0])*lowernumberratio)<part_points[:, 1])&(part_points[:, 1] < endpoints[1]-(endpoints[1]-endpoints[0])*uppernumberratio)]
        part_colors = part_colors[((endpoints[0]+(endpoints[1]-endpoints[0])*lowernumberratio)<part_points2[:, 1])&(part_points2[:, 1] < endpoints[1]-(endpoints[1]-endpoints[0])*uppernumberratio)]
        
        part_pcd = o3d.geometry.PointCloud()
        
        #count the number of white points, and check the ratio of (white points/all points)
        white_points_num = np.count_nonzero(part_colors==[1,1,1])
        all_points_num = np.count_nonzero(part_colors)
        

        if all_points_num == 0 or white_points_num/all_points_num < 0.04:
            directory = discard_dir
        else :#current optimal number for ratio of white points
            directory = output_dir
        

        part_pcd.points = o3d.utility.Vector3dVector(part_points)
        part_pcd.colors = o3d.utility.Vector3dVector(part_colors)  # Preserve the original colors

        distances = pcd.compute_nearest_neighbor_distance()
        avg_dist = np.mean(distances)

        cl, ind = part_pcd.remove_radius_outlier(nb_points=8, radius=4*avg_dist)  # Adjust nb_points and radius based on the data
        inlier_cloud = part_pcd.select_by_index(ind)

        part_clouds.append(inlier_cloud)

        #output_filename = f"{directory}/part_{i + 1}.pcd"
        
        #print(output_filename)
        # Save the part to a PCD file if needed
        #o3d.io.write_point_cloud(output_filename, part_pcd)
        #print(f"Saved: {output_filename}")

# Draw the parts with original colors
o3d.visualization.draw_geometries([part_clouds[0]], window_name="Partitioned Point Cloud Parts with Original Colors", width=800, height=600)


original chips color number is:  (3268567, 3)
original chips points number is:  (3268567, 3)
midpoints [-200568. -150832. -101096.  -51360.   -1624.]
